In [19]:
import pandas as pd
import plotly.graph_objects as go

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

In [20]:
# portfolio of index ETFs and their names
ticker_indexes_map = {
    "^GSPC": "S&P 500",
    "^IXIC": "NASDAQ 100",
    "^DJI": "Dow Jones Industrial Average",
    "^RUT": "Russell 2000",
    '^N225': "Nikkei 225",
    '^FTSE': "FTSE 100",
    '^FCHI': "CAC 40",
    '^GDAXI': "DAX",
    '^N225': "Nikkei 225",
    '^HSI': "Hang Seng Index",
}

tickers = list(ticker_indexes_map.keys())
index_labels = {ticker: f"{ticker} - {name}" for ticker, name in ticker_indexes_map.items()}

In [ ]:
# start_date = "2023-01-01"
# end_date = dt.datetime.now().strftime("%Y-%m-%d")

# # Download historical data from yf API
# data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# # download data to csv, filename have tickers joined by underscore and end_date
# data.to_csv("_".join(tickers) + "_" + end_date + ".csv")

[*********************100%***********************]  9 of 9 completed


In [22]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("_".join(tickers) + "_" + end_date + ".csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# set index as datetime
df.index = pd.to_datetime(df.index)

# keep only level 1 'Close' prices
df = df.xs('Close', level=1, axis=1)

# use descriptive names in charts/legends
df = df.rename(columns=index_labels)

# Show the result
df.head()

Ticker,^GSPC - S&P 500,^FTSE - FTSE 100,^GDAXI - DAX,^N225 - Nikkei 225,^IXIC - NASDAQ 100,^FCHI - CAC 40,^RUT - Russell 2000,^DJI - Dow Jones Industrial Average,^HSI - Hang Seng Index
Date,,,,,,,,,
2023-01-02,NaN,NaN,14069.259766,NaN,NaN,6594.569824,NaN,NaN,NaN
2023-01-03,3824.139893,7554.100098,14181.669922,NaN,10386.980469,6623.890137,1750.729980,33136.371094,20145.289062
2023-01-04,3852.969971,7585.200195,14490.780273,25716.859375,10458.759766,6776.430176,1772.540039,33269.769531,20793.109375
2023-01-05,3808.100098,7633.500000,14436.309570,25820.800781,10305.240234,6761.500000,1753.189941,32930.078125,21052.169922
2023-01-06,3895.080078,7699.500000,14610.019531,25973.849609,10569.290039,6860.950195,1792.800049,33630.609375,20991.640625


In [23]:
# date filter
# keep only last year of data
one_year_ago = dt.datetime.now() - dt.timedelta(days=365)
df = df[df.index >= one_year_ago]

In [24]:
# df copies for last 6 months and 3 months of data
df_6m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=182))]
df_3m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=91))]

In [25]:
# calculate simple returns with pct_change()
simple_returns = df.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding)
cumprod_simple = (1 + simple_returns).cumprod() - 1

In [26]:
# calculate simple returns with pct_change() for 6m and 3m dataframes
simple_returns_6m = df_6m.pct_change().fillna(0)
simple_returns_3m = df_3m.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding) for 6m and 3m dataframes
cumprod_simple_6m = (1 + simple_returns_6m).cumprod() - 1
cumprod_simple_3m = (1 + simple_returns_3m).cumprod() - 1

In [27]:
# plot cumprod_simple
fig = go.Figure()
for column in cumprod_simple.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple.index,
            y=cumprod_simple[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="Cumulative Simple Returns (Last Year)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [28]:
# Bar plot of final cumulative returns for each index
final_returns = cumprod_simple.iloc[-1].sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns.values,
        y=final_returns.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="Cumulative Returns (Last Year)",
    xaxis_title="Cumulative Return",
    yaxis_title="Index",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=500,
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [29]:
# plot last 6 months only
cumprod_simple_last_6_months = cumprod_simple.last('6M')
fig = go.Figure()
for column in cumprod_simple_last_6_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_6_months.index,
            y=cumprod_simple_last_6_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [30]:
# Bar plot of final cumulative returns for each index (Last 6 Months)
final_returns_6m = cumprod_simple_6m.iloc[-1].sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_6m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_6m.values,
        y=final_returns_6m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_6m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="Cumulative Returns (Last 6 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Index",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=500,
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [31]:
# plot last 3 months only
cumprod_simple_last_3_months = cumprod_simple.last('3M')
fig = go.Figure()
for column in cumprod_simple_last_3_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_3_months.index,
            y=cumprod_simple_last_3_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [32]:
# Bar plot of final cumulative returns for each index (Last 3 Months)
final_returns_3m = cumprod_simple_3m.iloc[-1].sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_3m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_3m.values,
        y=final_returns_3m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_3m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="Cumulative Returns (Last 3 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Index",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=500,
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [33]:
# plot last 6 months of cumprod_simple_6m returns 
fig = go.Figure()
for column in cumprod_simple_6m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_6m.index,
            y=cumprod_simple_6m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [34]:
# # Bar plot of final cumulative returns for each index (Last 6 Months - from df_6m)
# final_returns_6m_v2 = cumprod_simple_6m.iloc[-1].sort_values()

# # Create colors based on positive/negative returns
# colors = ['green' if x >= 0 else 'red' for x in final_returns_6m_v2.values]

# fig = go.Figure()
# fig.add_trace(
#     go.Bar(
#         x=final_returns_6m_v2.values,
#         y=final_returns_6m_v2.index,
#         orientation='h',
#         marker_color=colors,
#         text=[f"{x:.1%}" for x in final_returns_6m_v2.values],
#         textposition='outside',
#     )
# )

# fig.update_layout(
#     title="S&P 500 indexs - Cumulative Returns (Last 6 Months)",
#     xaxis_title="Cumulative Return",
#     yaxis_title="index",
#     template="plotly_white",
#     xaxis_tickformat=".0%",
#     height=500,
#     margin=dict(l=200, r=80, t=80, b=60),
# )
# fig.show()

In [35]:
# plot last 3 months of cumprod_simple_3m returns 
fig = go.Figure()
for column in cumprod_simple_3m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_3m.index,
            y=cumprod_simple_3m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [36]:
# # Bar plot of final cumulative returns for each index (Last 3 Months - from df_3m)
# final_returns_3m_v2 = cumprod_simple_3m.iloc[-1].sort_values()

# # Create colors based on positive/negative returns
# colors = ['green' if x >= 0 else 'red' for x in final_returns_3m_v2.values]

# fig = go.Figure()
# fig.add_trace(
#     go.Bar(
#         x=final_returns_3m_v2.values,
#         y=final_returns_3m_v2.index,
#         orientation='h',
#         marker_color=colors,
#         text=[f"{x:.1%}" for x in final_returns_3m_v2.values],
#         textposition='outside',
#     )
# )

# fig.update_layout(
#     title="S&P 500 indexs - Cumulative Returns (Last 3 Months)",
#     xaxis_title="Cumulative Return",
#     yaxis_title="index",
#     template="plotly_white",
#     xaxis_tickformat=".0%",
#     height=500,
#     margin=dict(l=200, r=80, t=80, b=60),
# )
# fig.show()